# Robustness Evaluation: Multi-Seed Aggregation for PETS vs MBPO
**How to use**
1) Place your per-seed CSVs in `/mnt/data/` with the following names:
- PETS policy logs: `policy_logs_pets_seed{seed}.csv`
- PETS events:      `events_pets_seed{seed}.csv`
- MBPO policy logs: `policy_logs_mbpo_seed{seed}.csv`
- MBPO events:      `events_mbpo_seed{seed}.csv`

2) If you only have a single-seed run, you may still run this notebook; it will compute bootstrap CIs across learners and provide a stability note.

**Metrics reported**: Time-to-Mastery (↓), Post-Content Gain (↑), Cumulative Reward (↑), Reward Variance (↓), Blueprint Deviation (↓)


In [1]:

import os, glob, json, numpy as np, pandas as pd
from pathlib import Path
from typing import Dict, List
import scipy.stats as st

data_dir = Path('/mnt/data')

# Discover seeds
pets_pl = sorted(data_dir.glob('policy_logs_pets_seed*.csv'))
mbpo_pl = sorted(data_dir.glob('policy_logs_mbpo_seed*.csv'))

def seeds_from_paths(paths):
    seeds = []
    for p in paths:
        s = int(p.stem.split('seed')[-1])
        seeds.append(s)
    return sorted(seeds)

pets_seeds = seeds_from_paths(pets_pl)
mbpo_seeds = seeds_from_paths(mbpo_pl)
seeds = sorted(set(pets_seeds).intersection(mbpo_seeds))
print(f"Found seeds -> PETS: {pets_seeds}, MBPO: {mbpo_seeds}, Intersect: {seeds}")

# Fallback: use single-seed (baseline) if multi-seed files are absent
single_seed_mode = False
if not seeds:
    print("No multi-seed files found; falling back to single-seed mode using policy_logs.csv and events.csv.")
    single_seed_mode = True


Found seeds -> PETS: [], MBPO: [], Intersect: []
No multi-seed files found; falling back to single-seed mode using policy_logs.csv and events.csv.


In [2]:

def time_to_mastery(events: pd.DataFrame, threshold=0.8) -> float:
    """
    Approximate time-to-mastery proxy using rolling correctness on same LO.
    Here we compute the median step index where a learner achieves >= threshold over a 5-item window.
    """
    events = events.copy()
    events['ts'] = pd.to_datetime(events['ts'])
    events = events.sort_values('ts')
    ans = events[events['type']=='answered'].copy()
    if ans.empty:
        return np.nan
    ans['correct'] = ans['payload_json'].apply(lambda s: json.loads(s)['correct']).astype(int)
    ans['step'] = np.arange(len(ans))
    # Rolling mean correctness (window=5)
    ans['roll5'] = ans['correct'].rolling(5, min_periods=5).mean()
    reached = ans[ans['roll5']>=threshold]
    if reached.empty:
        return np.nan
    return reached['step'].min()

def post_content_gain(policy: pd.DataFrame, events: pd.DataFrame, k=3) -> float:
    """
    Measure avg correctness over next-k answers after a content recommendation; minus prior 3 correctness.
    """
    events = events.copy()
    events['ts'] = pd.to_datetime(events['ts'])
    policy = policy.copy()
    policy['ts'] = pd.to_datetime(policy['ts'])
    
    def next_k_correct_fraction(row, k=3):
        sub = events[(events['learner_id']==row['learner_id']) & (events['lo_id']==row['lo_id']) & (events['ts']>row['ts'])]
        sub = sub[sub['type']=='answered'].head(k)
        if len(sub)==0: 
            return np.nan
        corr = sub['payload_json'].apply(lambda s: json.loads(s)['correct']).astype(int).mean()
        return corr

    def prev_k_correct_fraction(row, k=3):
        sub = events[(events['learner_id']==row['learner_id']) & (events['lo_id']==row['lo_id']) & (events['ts']<row['ts'])]
        sub = sub[sub['type']=='answered'].tail(k)
        if len(sub)==0: 
            return np.nan
        corr = sub['payload_json'].apply(lambda s: json.loads(s)['correct']).astype(int).mean()
        return corr

    # only rows where action is a content id
    policy_c = policy.dropna(subset=['action_id']).copy()
    policy_c['nextk'] = policy_c.apply(next_k_correct_fraction, axis=1, k=k)
    policy_c['prevk'] = policy_c.apply(prev_k_correct_fraction, axis=1, k=k)
    gain = (policy_c['nextk'] - policy_c['prevk']).dropna()
    return float(gain.mean()) if len(gain)>0 else np.nan

def cumulative_reward(policy: pd.DataFrame) -> float:
    return float(policy['reward'].sum())

def reward_variance(policy: pd.DataFrame) -> float:
    return float(policy['reward'].var())

def blueprint_deviation(events: pd.DataFrame, target=(0.2,0.6,0.2)) -> float:
    ev = events[events['type']=='question_shown'].copy()
    if ev.empty: return np.nan
    ev['difficulty'] = ev['payload_json'].apply(lambda s: json.loads(s)['difficulty'])
    obs = ev['difficulty'].value_counts(normalize=True)
    obs_vec = np.array([obs.get('Easy',0.0), obs.get('Medium',0.0), obs.get('Hard',0.0)])
    tgt = np.array(target)
    return float(np.abs(obs_vec - tgt).sum())  # L1 deviation


In [3]:

def compute_metrics_for_seed(policy_path: str, events_path: str) -> dict:
    pol = pd.read_csv(policy_path)
    ev  = pd.read_csv(events_path)
    # compute metrics
    ttm = time_to_mastery(ev)
    pcg = post_content_gain(pol, ev, k=3)
    cre = cumulative_reward(pol)
    rvar= reward_variance(pol)
    bdev= blueprint_deviation(ev)
    return {'time_to_mastery': ttm, 'post_content_gain': pcg, 'cum_reward': cre, 'reward_var': rvar, 'blueprint_dev': bdev}

def agg_stats(arr: np.ndarray) -> dict:
    arr = np.array(arr, dtype=float)
    arr = arr[~np.isnan(arr)]
    if len(arr)==0:
        return {'mean': np.nan, 'sd': np.nan, 'n': 0, 'ci_low': np.nan, 'ci_high': np.nan}
    mean = float(arr.mean())
    sd   = float(arr.std(ddof=1)) if len(arr)>1 else 0.0
    n    = int(len(arr))
    se   = sd/np.sqrt(n) if n>1 else 0.0
    ci   = (mean - 1.96*se, mean + 1.96*se)
    return {'mean': mean, 'sd': sd, 'n': n, 'ci_low': ci[0], 'ci_high': ci[1]}


In [4]:

results = []

if not single_seed_mode:
    for s in seeds:
        pets_pol = data_dir / f'policy_logs_pets_seed{s}.csv'
        pets_ev  = data_dir / f'events_pets_seed{s}.csv'
        mbpo_pol = data_dir / f'policy_logs_mbpo_seed{s}.csv'
        mbpo_ev  = data_dir / f'events_mbpo_seed{s}.csv'
        mp = compute_metrics_for_seed(pets_pol, pets_ev)
        mm = compute_metrics_for_seed(mbpo_pol, mbpo_ev)
        mp['alg']='PETS'; mp['seed']=s
        mm['alg']='MBPO'; mm['seed']=s
        results.extend([mp,mm])
else:
    # single-seed fallback using baseline names
    pets_pol = data_dir / 'policy_logs.csv'
    pets_ev  = data_dir / 'events.csv'
    if pets_pol.exists() and pets_ev.exists():
        mp = compute_metrics_for_seed(pets_pol, pets_ev)
        mp['alg']='BASELINE'; mp['seed']=0
        results.append(mp)

res = pd.DataFrame(results)
res

""


In [5]:

# Paired comparison if we have both algorithms
if not single_seed_mode and len(res['seed'].unique())>0:
    rows = []
    for metric in ['time_to_mastery','post_content_gain','cum_reward','reward_var','blueprint_dev']:
        wide = res.pivot(index='seed', columns='alg', values=metric).dropna()
        if {'PETS','MBPO'}.issubset(wide.columns):
            diff = wide['MBPO'] - wide['PETS']
            mean = diff.mean(); sd = diff.std(ddof=1); n=len(diff); se = sd/np.sqrt(n) if n>1 else 0.0
            t,p = st.ttest_rel(wide['MBPO'], wide['PETS']) if n>1 else (np.nan, np.nan)
            d = mean / sd if (sd>1e-9) else np.nan  # Cohen's d (paired)
            ci = (mean - 1.96*se, mean + 1.96*se)
            rows.append({'metric':metric, 'mean_diff':mean, 'ci_low':ci[0], 'ci_high':ci[1], 'p_value':p, 'cohen_d':d, 'n':n})
    stats_df = pd.DataFrame(rows)
    display(stats_df)
else:
    print("Paired stats unavailable (need multi-seed PETS & MBPO).")


Paired stats unavailable (need multi-seed PETS & MBPO).


In [6]:

# Aggregate table (mean±sd with 95% CI) per algorithm
if len(res)>0:
    rows = []
    for alg in sorted(res['alg'].unique()):
        sub = res[res['alg']==alg]
        for metric in ['time_to_mastery','post_content_gain','cum_reward','reward_var','blueprint_dev']:
            st_ = agg_stats(sub[metric].values)
            rows.append({'alg':alg,'metric':metric, **st_})
    agg_df = pd.DataFrame(rows)
    display(agg_df)


In [7]:

# Export LaTeX table summarizing mean±sd and 95% CI
if len(res)>0:
    def fmt(m,sd,lo,hi):
        if np.isnan(m): return '--'
        return f"{m:.3f} ± {sd:.3f} (CI {lo:.3f},{hi:.3f})"
    tbl = agg_df.pivot(index='metric', columns='alg', values=['mean','sd','ci_low','ci_high'])
    lines = []
    lines.append('\\begin{table}[t]')
    lines.append('\\centering')
    lines.append('\\caption{Multi-Seed Robustness Summary (mean±sd; 95\% CI)}')
    lines.append('\\begin{tabular}{lcc}')
    lines.append('\\toprule')
    lines.append('Metric & PETS & MBPO \\')
    lines.append('\\midrule')
    for metric in ['time_to_mastery','post_content_gain','cum_reward','reward_var','blueprint_dev']:
        row = []
        row.append(metric.replace('_','\\_'))
        for alg in ['PETS','MBPO']:
            try:
                m = tbl['mean'][alg][metric]; sd = tbl['sd'][alg][metric]; lo = tbl['ci_low'][alg][metric]; hi = tbl['ci_high'][alg][metric]
                row.append(fmt(m,sd,lo,hi))
            except Exception:
                row.append('--')
        lines.append(' & '.join(row) + ' \\')
    lines.append('\\bottomrule')
    lines.append('\\end{tabular}')
    lines.append('\\label{tab:robustness}')
    lines.append('\\end{table}')
    tex = '\n'.join(lines)
    with open('/mnt/data/robustness_table.tex','w') as f:
        f.write(tex)
    print('Wrote LaTeX table to /mnt/data/robustness_table.tex')


<>:10: SyntaxWarning: invalid escape sequence '\%'
<>:10: SyntaxWarning: invalid escape sequence '\%'
/tmp/ipykernel_7829/664589677.py:10: SyntaxWarning: invalid escape sequence '\%'
  lines.append('\\caption{Multi-Seed Robustness Summary (mean±sd; 95\% CI)}')
